# Python En Parrellèle

Avant de commencer à vous expliquer comment faire du threading et du multiprocessing en python, vous devez avoir une
idée de base de comment fonctionne votre ordinateur. Sans ces connaissances de base, il sera vraiment difficile de
comprendre ce que vous codez.

Un CPU est en ensemble de Core. Un core est un unité de calcul ou “processing unit”. C’est essentiellement une petite
calculatrice qui fait des petites opérations vraiment simples et qui ne peut seulement faire un seul petit calcul à la
fois. Ça veut dire que si votre CPU possède 9 cores comme dans la figure, alors votre CPU peut faire seulement 9
opérations à la fois. Évidemment, vous devez vous doutez que vous arrivez à demander à votre CPU de faire plus que 9
tâches à fois… Sinon comment pourriez vous faire pour être sur discord avec vos en même temps que de jouer à Valheim
sur le serveur où vous êtes host toute en écoutant votre cours sur zoom… Eh bien, c’est l’utilitée des threads. Un
thread est un programme ou un ensemble d’opérations qu’on demande à un core d’exécuter. Afin qu’un core soit en mesure
d’exécuter les opérations nécessaire à plusieurs programmes à la fois, il alterne entre ceux-ci en exécutant une partie
de chaque programme. Il peut donc y avoir plusieurs threads sur un seul core et les programmes sembleront s'exécuter en
même temps. Toutefois, il ne faut pas se leurrer. En ayant deux programmes roulant sur le même core, le temps
d'exécution sera plus grand ou égale au temps que ça aurait prit pour lancer ces deux programmes de façon séquentiel.
Par contre, si on lance ces deux même threads sur deux cores différent, ce qu’on appelle le multiprocessing, les deux
programmes se feront réellement en même temps et donc le temps d’exécution finale sera égale au temps du programme le
plus long.

Dans cette présentation, je vais vous montrer comment lancer plusieurs threads en même temps sur un seul core. Par
après nous allons voir comment lancer des programmes sur plusieurs cores différents afin d’avoir des programmes
s’exécutant réellement en parallèles dans l’objectif de sauver du temps.


![CPU_Shm](figures/CPU_Shm.png)

In [23]:
import time
import functools
import numpy as np
import multiprocessing as mp
from scipy.signal import convolve2d
import psutil
import threading as th

In [24]:
def benchmark(_func=None, *, box_length=50):
    def decorator_print_func_section(func):
        box_char = '-'
        title = (box_char*box_length) + ' ' + func.__name__ + ' ' + (box_char*box_length)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(title)
            start_time = time.time()
            out = func(*args, **kwargs)
            stats_line = f"Executed in {time.time() - start_time:.5f}s."
            print(stats_line)
            print(box_char*len(title))
            return out
        return wrapper

    if _func is None:
        return decorator_print_func_section
    else:
        return decorator_print_func_section(_func)

In [25]:
def split(container, count):
    return [container[i::count] for i in range(count)]

def apply_intern(M, kernels):
    return [convolve2d(M, k) for k in kernels]

@benchmark
def apply_kernels_on_matrix_sp(M, kernels):
    return apply_intern(M, kernels)

@benchmark
def apply_kernels_on_matrix_mp(M, kernels):
    cpu_count = psutil.cpu_count(logical=False)
    with mp.Pool(cpu_count) as pool:
        out = pool.starmap(convolve2d, [(M, k) for k in kernels])
    return out

@benchmark
def apply_kernels_on_matrix_mt(M, kernels):
    thread_count = psutil.cpu_count(logical=False)
    splitter_kernels = split(kernels, thread_count)
    threads = [th.Thread(target=apply_intern, args=(M, splitter_kernels[i])) for i in range(thread_count)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

In [26]:
if __name__ == '__main__':
    A = np.random.randint(0, 100, (1_000, 1_000))
    kernels = [np.random.randint(0, 100, (10, 10)) for _ in  range(100)]

    apply_kernels_on_matrix_sp(A, kernels)
    apply_kernels_on_matrix_mp(A, kernels)
    apply_kernels_on_matrix_mt(A, kernels)

-------------------------------------------------- apply_kernels_on_matrix_sp --------------------------------------------------
Executed in 18.76574s.
--------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- apply_kernels_on_matrix_mp --------------------------------------------------
Executed in 6.41115s.
--------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- apply_kernels_on_matrix_mt --------------------------------------------------
Executed in 18.89957s.
--------------------------------------------------------------------------------------------------------------------------------


Thread

In [12]:
def sleepy_boiiii(id_):
    print(f"{id_}: au dodo")
    time.sleep(2.5)
    print(f"{id_}: au réveil")

In [15]:
thread1 = th.Thread(target=sleepy_boiiii, args=(0,))
thread2 = th.Thread(target=sleepy_boiiii, args=(1,))
thread1.start()
thread2.start()
thread1.join()
thread2.join()

0: au dodo
1: au dodo
0: au réveil1: au réveil

2.508399248123169


In [18]:
def sleepy_boiiii_lock(id_, lock):
    lock.acquire()
    print(f"{id_}: au dodo")
    time.sleep(2.5)
    print(f"{id_}: au réveil")
    lock.release()

In [19]:
lock = th.Lock()
thread1 = th.Thread(target=sleepy_boiiii_lock, args=(0, lock))
thread2 = th.Thread(target=sleepy_boiiii_lock, args=(1, lock))
thread1.start()
thread2.start()
thread1.join()
thread2.join()

0: au dodo
0: au réveil
1: au dodo
1: au réveil


Classes qui dérivent de Process et Thread

In [21]:
class MonThread(th.Thread):

    def __init__(self, lock, func, *fargs, **fkwargs):
        super(MonThread, self).__init__()
        self._lock = lock
        self._func = func
        self._fargs = fargs
        self._fkwargs = fkwargs

    def run(self):
        with lock:
            retour = self._func(*self._fargs, **self._fkwargs)
        return retour

In [22]:
thread1 = MonThread(lock, sleepy_boiiii, 0)
thread2 = MonThread(lock, sleepy_boiiii, 1)
thread1.start()
thread2.start()
thread1.join()
thread2.join()

0: au dodo
0: au réveil
1: au dodo
1: au réveil
